##![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **Assignment 2 - Matrices & Graphs**
# **Matrix-Vector and Matrix-Matrix Multiplications**


#### Matrix-Vector Multiplication 

In [1]:
file1 = "a_100x200.txt"
file2 = "b_200x100.txt"
x=file2.replace(".txt","")
y=file1.replace(".txt","")
m=y.split("x")[0]
n=x.split("x")[0]
kval=x.split("x")[1]

isK=kval.find('K') 
#print isK

if(isK> 0):
   inter=kval.replace("K","")
   p=int(inter)*1000
else:
   p=int(kval)

print p
mat = sc.textFile("ass2Data/a_100x200.txt")
mat2 = sc.textFile("ass2Data/b_200x100.txt")
# mat = sc.textFile(file1)
# mat2 = sc.textFile(file2)
matFilter = mat.map(lambda x: [float(i) for i in x.split(" ")])
matFilter2 = mat2.map(lambda x: [float(i) for i in x.split(" ")])

In [2]:
matFilter = mat.map(lambda x: [float(i) for i in x.split(" ")])

In [3]:
matgroup = matFilter.map(lambda x: (x[0], [x[2]])).reduceByKey(lambda p,q: p+q)

In [227]:
def replaceText(line):
    line=line.replace("array([ ","");
    line=line.replace("])","");
    return float(line);
#vect = sc.textFile("x_200.txt").map(lambda x: (1, [replaceText(x)])).reduceByKey(lambda p,q: p+q)
vect = sc.textFile("ass2Data/x_200.txt").map(lambda x: (1, [replaceText(x)])).reduceByKey(lambda p,q: p+q)

In [138]:
import numpy as np
matvec = matgroup.cartesian(vect)
matvecmul = matvec.map(lambda x: (x[0][0], np.dot(x[0][1], x[1][1])))
matvecmul.take(2)

[(0.0, 51.456960943724127), (1.0, 51.609732508318054)]

In [140]:
matvecmul.saveAsTextFile("MatrixVectorMultiplication200.txt")

#### Block Matrix-Matrix Multiplication 

In [244]:
s = 10
t = 20 
v = 5

In [4]:
matgroupp = matFilter.map(lambda x: (x[0], [x[2]])).reduceByKey(lambda p,q: p+q)
matgroup2 = matFilter2.map(lambda x: (x[1], [x[2]])).reduceByKey(lambda p,q: p+q)
matInter = matgroupp.cartesian(matgroup2)
matmul = matInter.map(lambda x: ((x[0][0], x[1][0]), np.dot(x[0][1], x[1][1]))).sortByKey(True)

In [232]:
matmul.saveAsTextFile("MatrixMatrixMultiplicationBBB100_100.txt")

#### Matrix-Matrix Multiplication  using One Pass

In [32]:
p = 100
matOnePass = matFilter.flatMap(lambda x : [((x[0], k), (x)) for k in range(0, p)])
matOnePasslist = matOnePass.map(lambda x: (x[0], [x[1][2]])).reduceByKey(lambda p,q: p+q).sortByKey(True)

In [33]:
matOnePass2 = matFilter2.flatMap(lambda x : [((k, x[1]), (x)) for k in range(0, p)])
matOnePasslist2 = matOnePass2.map(lambda x: (x[0], [x[1][2]])).reduceByKey(lambda p,q: p+q).sortByKey(True)

In [34]:
matCartOP = matOnePasslist.join(matOnePasslist2).sortByKey(True)

In [36]:
matmul = matCartOP.map(lambda x: (x[0], np.dot(x[1][0], x[1][1]))).sortByKey(True)

In [38]:
matmul.saveAsTextFile("MatrixMatrixMultiplicationOnePass100_100.txt")

#### Another method of Block Matrix Multiplication

In [159]:
s = 10
t = 20
v = 5
matBlocks = matFilter.flatMap(lambda x : [((int(x[0]/s), int(x[1]/t), k), (int(x[0]%s), int(x[1]%t), x[2])) for k in range(0, p/v)])

In [160]:
matBlocks2 = matFilter2.flatMap(lambda x : [((k, int(x[0]/t), int(x[1]/v)), (int(x[0]%t), int(x[1]%v), x[2])) for k in range(0, m/s)])

In [103]:
matOnePassBlock = matBlocks.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda p,q: p+q).sortByKey(True)
matOnePassBlock2 =  matBlocks2.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda p,q: p+q).sortByKey(True)

In [106]:
matBlockCartOP = matOnePassBlock.join(matOnePassBlock2).sortByKey(True)

In [124]:
def multiply(x):
    listA = x[1][0]
    listB = x[1][1]
    fDict = {}
    for a in listA:
        for b in listB:
            print a, b
            if a[1] == b[0]:
                fDict[(a[0], b[1])] = fDict.get((a[0], b[1]),0)+a[2]*b[2]
    print fDict
    finalList = []
    for key, value in fDict.iteritems():
        finalList.append(((x[0][0]*s + key[0], x[0][2]*v + key[1]), value))
    return finalList

In [152]:
matBlockMul = matBlockCartOP.flatMap(multiply)

In [153]:
def secondPass(values):
    result = 0
    for value in values:
        result += value
    return result    

In [156]:
matSecPass = matBlockMul.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda p,q: p+q).sortByKey(True)
matFinalRes = matSecPass.map(lambda x: (x[0], secondPass(x[1]))).sortByKey(True)

In [141]:
# matBlockMul.saveAsTextFile("MatrixMatrixMultiplicationBlock100_100.txt")

In [158]:
matFinalRes.collect()

[((0, 0), 90.0),
 ((0, 1), 100.0),
 ((0, 2), 110.0),
 ((1, 0), 240.0),
 ((1, 1), 275.0),
 ((1, 2), 310.0)]

# Shallow Graphs

In [1]:
sgMat = sc.textFile("ass2Data/Assign2_200.txt")
sgMatFilter = sgMat.map(lambda x: [int(i) for i in x.split(" ")])

In [2]:
# intData = sgMatFilter.map(lambda x: [x[1], x[0], x[2]])
# sgMatFilter = sgMatFilter.union(intData)

In [4]:
sgFinal = sgMatFilter.map(lambda x: (x[0], {x[1]:x[2]}))\
                     .map(lambda x: (x[0], [x[1]]))\
                     .reduceByKey(lambda p,q: p+q)\
                     .sortByKey(True)

In [5]:
sgFinal2 = sgMatFilter.map(lambda x: (x[1], {x[0]: x[2]}))\
                      .map(lambda x: (x[0], [x[1]]))\
                      .reduceByKey(lambda p,q: p+q)\
                      .sortByKey(True) 

In [6]:
def comparator(list1, list2):
    sum = 0
    for x in list1:
        if x in list2:
            sum += 1
    return sum

In [7]:
sgJoined = sgFinal.cartesian(sgFinal2)

In [8]:
sgFinalMat = sgJoined.map(lambda x: ((x[0][0], x[1][0]), comparator(x[0][1], x[1][1])))

In [9]:
addData = sgFinalMat.filter(lambda x: x[1] != 0)

In [10]:
A = sgMatFilter.map(lambda x: ((x[0], x[1]), x[2]))

In [11]:
aSqrPlusA = addData.union(A)\
                   .map(lambda x: (x[0], x[1]))\
                   .reduceByKey(lambda p,q: p+q)\
                   .sortByKey(True)

In [12]:
aSqrPlusA.count()

1000000

In [452]:
if aSqrPlusA.count() != 10000:
    print "not Shallow"
else:
    print "Shallow"
aSqrPlusA.count()

not Shallow


10000

In [295]:
# def addition(tup):
#     if tup[0] in addData:
#         return 1
#     else:
#         return tup[1]
# finalResult = sgFinalMat.map(lambda x: (x[0], addition(x)))

In [503]:
# sgMat = sc.textFile("ass2Data/assign2_5_1.txt")
# sgMatFilter = sgMat.map(lambda x: [int(i) for i in x.split(" ")])
# # intData = sgMatFilter.map(lambda x: [x[1], x[0], x[2]])
# # sgMatFilter = sgMatFilter.union(intData)
# liste = [range(0,5), range(0, 5)]
# finalList = []
# import itertools
# for element in itertools.product(*liste):
#     finalList.append((element, 0))
# finalList[:10]

[((0, 0), 0),
 ((0, 1), 0),
 ((0, 2), 0),
 ((0, 3), 0),
 ((0, 4), 0),
 ((1, 0), 0),
 ((1, 1), 0),
 ((1, 2), 0),
 ((1, 3), 0),
 ((1, 4), 0)]

In [504]:
# wholeRDD = sc.parallelize(finalList)
# sgMatTupFilter = sgMatFilter.map(lambda x: ((x[0], x[1]), x[2]))
# matrixA = sgMatTupFilter.union(wholeRDD)\
#                       .map(lambda x: (x[0], x[1]))\
#                       .reduceByKey(lambda p,q: p+q)\
#                       .sortByKey(True)
# matrixA.take(10)

[((0, 0), 0),
 ((0, 1), 1),
 ((0, 2), 1),
 ((0, 3), 1),
 ((0, 4), 1),
 ((1, 0), 0),
 ((1, 1), 0),
 ((1, 2), 1),
 ((1, 3), 1),
 ((1, 4), 1)]

In [505]:
# sgMat = matrixA.map(lambda x: (x[0][0], [x[1]])).reduceByKey(lambda p,q: p+q)
# sgMat2 = matrixA.map(lambda x: (x[0][1], [x[1]])).reduceByKey(lambda p,q: p+q)
# matInter = sgMat.cartesian(sgMat2)
# matmul = matInter.map(lambda x: ((x[0][0], x[1][0]), np.dot(x[0][1], x[1][1]))).sortByKey(True)

In [507]:
# addData = matmul.filter(lambda x: x[1] != 0)
# addData.count()
# matmul.collect()

[((0, 0), 0),
 ((0, 1), 0),
 ((0, 2), 1),
 ((0, 3), 2),
 ((0, 4), 1),
 ((1, 0), 0),
 ((1, 1), 0),
 ((1, 2), 0),
 ((1, 3), 1),
 ((1, 4), 0),
 ((2, 0), 0),
 ((2, 1), 0),
 ((2, 2), 0),
 ((2, 3), 0),
 ((2, 4), 0),
 ((3, 0), 0),
 ((3, 1), 0),
 ((3, 2), 0),
 ((3, 3), 0),
 ((3, 4), 0),
 ((4, 0), 0),
 ((4, 1), 0),
 ((4, 2), 0),
 ((4, 3), 0),
 ((4, 4), 0)]